In [1]:
import numpy as np
import pandas as pd
from river import optim
from river import metrics
from river.drift.binary import EDDM
from sail.telemetry import TracingClient
from sail.models.auto_ml.tune import SAILTuneGridSearchCV
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.models.river.linear_model import LinearRegression
from sail.models.auto_ml.auto_pipeline import SAILAutoPipeline
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.drift_detection.drift_detector import SAILDriftDetector
from sail.transformers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
linear_reg = LinearRegression(optimizer=optim.SGD(0.1))
random_forest = AdaptiveRandomForestRegressor()

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", "passthrough"),
]
sail_pipeline = SAILPipeline(steps=steps, scoring=metrics.R2, verbosity_level=1, verbosity_interval=2)

#### HPO Params


In [5]:
params_grid = [
    {
        "regressor": [linear_reg],
        "regressor__l2": [0.1, 0.9],
        "regressor__intercept_init": [0.2, 0.5],
    },
    {
        "regressor": [random_forest],
        "regressor__n_models": [10, 15, 20]
    },
]

#### Create Model Instance


In [7]:
tracer = TracingClient(service_name="SAIL_Testing", otlp_endpoint="http://localhost:4318")

auto_pipeline = SAILAutoPipeline(
    pipeline=sail_pipeline,
    pipeline_params_grid=params_grid,
    search_method=SAILTuneGridSearchCV,
    search_method_params={
        "num_cpus_per_trial": 1,
        "max_iters": 1,
        "early_stopping": False,
        "mode": "max",
        "scoring": "r2",
        "pipeline_auto_early_stop": False,
        "keep_best_configurations": 2,
        # "runtime_env": {
        #     "py_modules": [sail],
        #     "pip": [
        #         "scikit-learn>=1.2",
        #         "logzero",
        #         "numpy>=1.23",
        #         "river==0.14.*",
        #         "ray>=2.5",
        #         "tune_sklearn",
        #         "dill",
        #         "ipython",
        #     ],
        # },
        # "namespace": "sail-tune",
        # "cluster_address": "ray://localhost:10001"
    },
    search_data_size=500,
    incremental_training=True,
    drift_detector=SAILDriftDetector(model=EDDM(), drift_param="difference"),
    pipeline_strategy="DetectAndIncrement",
    verbosity_level=1, 
    verbosity_interval=2,
    tensorboard_log_dir=None,
    tracer=tracer
)

[2023-09-20 01:27:02:55] - INFO - SAIL (PipelineStrategy) - Pipeline Strategy [DetectAndIncrement] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'SCORE_AND_DETECT_DRIFT', 'PARTIAL_FIT_MODEL']


#### Start Training


In [8]:
y_preds = []
y_true = []
batch_size = 50

In [9]:
batch_size = 50
with tracer.trace_as_current_span(span_name="Pipeline-Train"):
    for start in range(0, 1000, batch_size):

        end = start + batch_size

        X_train = X.iloc[start:end]
        y_train = y.iloc[start:end]

        if end > 500: # search_data_size is 500
            preds = auto_pipeline.predict(X_train)
            y_preds.extend(list(preds))
            y_true.extend(list(y_train))

        auto_pipeline.train(X_train, y_train)

2023-09-20 01:27:16,259	INFO tune.py:1148 -- Total run time: 11.19 seconds (11.18 seconds for the tuning loop).



>>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------


SAIL Pipeline fit: 100%========== [Steps: 3/3, ETA: 00:01<00:00, Elapsed:1.440s, Batch Size=500, Score=0.726]            
[2023-09-20 01:27:19:309] - INFO - SAIL (PipelineStrategy) - Pipeline tuning completed. Disconnecting Ray cluster...
[2023-09-20 01:27:19:310] - INFO - SAIL (PipelineStrategy) - Found best params: {'regressor': AdaptiveRandomForestRegressor(drift_detector=ADWIN (
      delta=0.001
      clock=32
      max_buckets=5
      min_window_length=5
      grace_period=10
    ),
                                  metric=MSE: 0., n_models=20,
                                  warning_detector=ADWIN (
      delta=0.01
      clock=32
      max_buckets=5
      min_window_length=5
      grace_period=10
    )), 'regressor__n_models': 20}



>>> Epoch: 12 | Samples Seen: 550 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0014s, Metric=R2, Batch Size=50, P_Score=0.586]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0011s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.177s, Batch Size=50, P_Score=0.586]            


...

SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0012s, Metric=R2, Batch Size=50, P_Score=0.604]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0008s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.178s, Batch Size=50, P_Score=0.604]            



>>> Epoch: 18 | Samples Seen: 850 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0013s, Metric=R2, Batch Size=50, P_Score=0.612]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0008s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.208s, Batch Size=50, P_Score=0.612]            



>>> Epoch: 20 | Samples Seen: 950 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0013s, Metric=R2, Batch Size=50, P_Score=0.636]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0009s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.194s, Batch Size=50, P_Score=0.636]            


#### Save Model


In [9]:
auto_pipeline.save_model(".")

'./sail_auto_pipeline'

#### Load Model


In [10]:
new_auto_pipeline = SAILAutoPipeline.load_model(".")

[2023-09-12 00:47:55:862] - INFO - SAIL (PipelineStrategy) - Sending training output to Tensorboard logs. Please run `tensorboard --logdir /Users/dhaval/Projects/SAIL/sail/LOGS` in terminal to check training progress.
[2023-09-12 00:47:55:863] - INFO - SAIL (PipelineStrategy) - Pipeline Strategy [DetectAndIncrement] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'SCORE_AND_DETECT_DRIFT', 'PARTIAL_FIT_MODEL']
SAIL Pipeline Progressive Score: 100%========== [Points: 2000/2000, Elapsed:0.0042s, Metric=R2, Batch Size=2000, P_Score=0.671]


#### Continue training using the load model


In [11]:
for start in range(1501, 2001, batch_size):

    end = start + batch_size

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    preds = new_auto_pipeline.predict(X_train)
    y_preds.extend(list(preds))
    y_true.extend(list(y_train))

    new_auto_pipeline.train(X_train, y_train)


>>> Epoch: 32 | Samples Seen: 1550 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0089s, Metric=R2, Batch Size=50, P_Score=0.677]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0020s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.127s, Batch Size=50, P_Score=0.677]            



>>> Epoch: 36 | Samples Seen: 1750 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0017s, Metric=R2, Batch Size=50, P_Score=0.693]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0010s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.138s, Batch Size=50, P_Score=0.693]            



>>> Epoch: 40 | Samples Seen: 1950 -------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0015s, Metric=R2, Batch Size=50, P_Score=0.705]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0011s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.131s, Batch Size=50, P_Score=0.705]            


#### Final Score


In [12]:
new_auto_pipeline.progressive_score

0.7048665323702248

### Plot predictions


In [13]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(500)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()